In [ ]:
# import pandas as pd
# import numpy as np
# import os, fnmatch
# import re
# import random
# import glob
# import cv2
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib
# from matplotlib import font_manager
# import warnings
# from google.colab import drive
# from tqdm.auto import tqdm
# from sklearn.model_selection import train_test_split
# import argparse

# from PIL import Image

# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import f1_score

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import ConcatDataset, Dataset, DataLoader, Subset
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2
# from torchvision.transforms.functional import to_tensor
# from torchvision.transforms import ToTensor
# import torchvision
# #import torchvision.transforms as T
# import torchvision.models as models
# import torchvision.transforms as transforms
# #import torchvision.datasets as datasets

# from torchvision.utils import save_image

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import matplotlib.pyplot as plt
import itertools
import collections
import gc

import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import torchvision
from torch.cuda.amp import GradScaler, autocast

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
#os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1, 2, 3'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

In [ ]:
CFG = {
    'SEED':41,
    'IMG_SIZE':224,
    'BATCH_SIZE': 128, # 128
    'EPOCHS':30,
    'LEARNING_RATE': 3e-4 #3e-4
}

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED'])

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# dataset_dir = '/content/drive/MyDrive/open/'
# train_folder = os.listdir(dataset_dir + 'train')

# train_path = []
# for folder in train_folder:
#   temp = os.path.join(dataset_dir + 'train/' + folder)
#   if os.path.isdir(temp):
#     train_path += glob.glob(temp + '/*')

In [ ]:
dataset_dir = '/content/drive/MyDrive/open (3)/'

In [ ]:
all_img_list = glob.glob(dataset_dir + 'train/*/*')

In [ ]:

train_df = pd.DataFrame(columns=['img_path', 'label'])
train_df['img_path'] = all_img_list
train_df['label'] = train_df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [ ]:
# train_df = pd.DataFrame(train_path, columns = ['img_path'])
# train_df['label'] = train_df.img_path.apply(lambda x: str(x).split('/')[-2])

In [ ]:
#le = LabelEncoder()
#train_df['label'] = le.fit_transform(train_df['label'])
# test_df

labels_kr = ['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점', '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스', '훼손']
indexes = sorted(train_df.label.unique())
mapping_dict = dict(zip(indexes, labels_kr))
train_df['label'] = train_df['label'].map(mapping_dict)

In [ ]:
uniques = np.unique(train_df['label'])
uniques

array(['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점',
       '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스',
       '훼손'], dtype=object)

In [ ]:
# train_df['label'].value_counts()

In [ ]:
# def plot_images(df, label):
#   plt.figure(figsize = (12,12))
#   temp = df[df['label'] == label]
#   print(label)
#   temphead = temp['img_path'].head(20)
#   for idx, img_path in enumerate(temphead):
#     plt.subplot(4, 5, idx + 1)
#     img = Image.open(img_path)
#     img = img.resize((224, 224))
#     plt.imshow(img)
#     plt.axis('off')
#   plt.tight_layout()
#   plt.show()

# df = train_df.copy()
# for label in df.label.unique():
#   plot_images(df, label)

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
# plt.rc('font', family = 'NanumBarunGothic')

In [ ]:
# axes = sns.histplot(y = 'label', hue = 'label', data = train_df, stat = 'count')
# for container in axes.containers:
#   values = container.datavalues.tolist()
#   x = max(values)
#   y = values.index(x)
#   axes.text(x, y, x)
# plt.show()

In [ ]:
# train, val, _, _ = train_test_split(train_df, train_df['label'], test_size=0.3, stratify=train_df['label'], random_state=CFG['SEED'])

In [ ]:
# le = preprocessing.LabelEncoder()
# train['label'] = le.fit_transform(train['label'])
# val['label'] = le.transform(val['label'])

In [ ]:
#le = preprocessing.LabelEncoder()
#train_df['label'] = le.fit_transform(train_df['label'])


# initialising dict
indexes = range(0, len(uniques))
init_dict = dict(zip(labels_kr, indexes))
train_df['label'] = train_df['label'].map(init_dict)

# inverse dict
inverse_dict = {v: k for k, v in init_dict.items()}

In [ ]:
# def get_mean_std(data_dir):
#   transform = transforms.Compose([
#       transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
#       transforms.ToTensor()
#   ])

#   dataset = torchvision.datasets.ImageFolder(os.path.join(f'{data_dir}'), transform = transform)

#   meanRGB = [np.mean(x.numpy(), axis = (1, 2)) for x, _ in dataset]
#   stdRGB = [np.std(x.numpy(), axis = (1, 2)) for x, _ in dataset]

#   meanR = np.mean([mean[0] for mean in meanRGB])
#   meanG = np.mean([mean[1] for mean in meanRGB])
#   meanB = np.mean([mean[2] for mean in meanRGB])

#   stdR = np.mean([std[0] for std in stdRGB])
#   stdG = np.mean([std[1] for std in stdRGB])
#   stdB = np.mean([std[2] for std in stdRGB])

#   print('Mean: ', meanR, meanG, meanB)
#   print('Std: ', stdR, stdG, stdB)

#   return meanR, meanG, meanB, stdR, stdG, stdB

def get_mean_std(data_dir):
    transform = transforms.Compose([
        transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
        transforms.ToTensor()
    ])

    dataset = torchvision.datasets.ImageFolder(data_dir, transform=transform)

    images = [x for x, _ in dataset]
    images = torch.stack(images, dim=0)

    meanRGB = torch.mean(images, dim=(0, 2, 3)).numpy()
    stdRGB = torch.std(images, dim=(0, 2, 3)).numpy()

    meanR, meanG, meanB = meanRGB
    stdR, stdG, stdB = stdRGB

    print('Mean: ', meanR, meanG, meanB)
    print('Std: ', stdR, stdG, stdB)

    return meanR, meanG, meanB, stdR, stdG, stdB

TrainSetDir = dataset_dir
#meanR, meanG, meanB, stdR, stdG, stdB = get_mean_std(TrainSetDir)
meanR, meanG, meanB, stdR, stdG, stdB = 0.6085033, 0.599786, 0.58327097, 0.13209724, 0.13448739, 0.14057396

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, img_path_list, label_list, transforms=None):
#       self.img_path_list = img_path_list
#       self.label_list = label_list
#       self.transforms = transforms

        
#     def __getitem__(self, index):
#       img_path = self.img_path_list[index]
        
#       image = cv2.imread(img_path)
        
#       if self.transforms is not None:
#         image = self.transforms(image=image)['image']
        
#       if self.label_list is not None:
#         label = self.label_list[index]
#         return image, label
        
#       else:
#         return image
        
#     def __len__(self):
#       return len(self.img_path_list)

['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점',
       '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스',
       '훼손']

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
      self.img_path_list = img_path_list
      self.label_list = label_list
      self.transforms = transforms
      self.another_transform = A.Compose([
          A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
          # A.HorizontalFlip(),
          # A.OneOf([
          #     A.CLAHE(clip_limit=2),
          #     A.RandomBrightnessContrast(),
          # ], p=0.5),
          A.HorizontalFlip(),
          A.CLAHE(clip_limit=2),
          A.RandomBrightnessContrast(),
          A.ColorJitter(),
          A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
          ToTensorV2()
      ])
        
    def __getitem__(self, index):
      img_path = self.img_path_list[index]
        
      image = cv2.imread(img_path)
      #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
      if self.label_list is not None:
        label = self.label_list[index]
        
        if self.transforms is not None:
          # 걸레받이수정, 꼬임, 몰딩수정 - rotate, verticalflip x
          if label in [1, 3, 7]:
            image = self.another_transform(image=image)['image']
          else:
            image = self.transforms(image=image)['image']
        
        return image, label

      else:
        if self.transforms is not None:
          image = self.transforms(image=image)['image']
          
      return image
        
    def __len__(self):
      return len(self.img_path_list)

In [ ]:
#['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점',
#       '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스',
#       '훼손']

#이음부불량 창틀문틀수정 - 세로
#곰팡이, 녹오염, 반점, 오염 - 샤프닝, 라이트, 컬러지터, 랜덤감마

# 13, 14 (세로) VerticalFlip
# 1, 3, 7 (가로) HorizontalFlip
# 2 (하단) Shift
# 2, 4, 8, 10 - A.Sharpen(alpha=(0.5, 1.0), lightness=(0.5, 1.0), always_apply=True),
    #A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.5, always_apply=True),
    #A.RandomGamma(gamma_limit=(0.8, 1.2), always_apply=True),

      # self.another_transform1314 = A.Compose([
      #     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
      #     A.VerticalFlip(),
      #     A.CLAHE(clip_limit=2),
      #     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
      #     #A.Sharpen(alpha=1, lightness=0),
      #     #A.ColorJitter(saturation=0.2, hue=0.1),
      #     A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
      #     ToTensorV2()
      # ])
      # self.another_transform137 = A.Compose([
      #     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
      #     A.HorizontalFlip(),
      #     A.CLAHE(clip_limit=2),
      #     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
      #     #A.Sharpen(alpha=1, lightness=0),
      #     #A.ColorJitter(saturation=0.2, hue=0.1),
      #     A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
      #     ToTensorV2()
      # ])
      # self.another_transform2 = A.Compose([
      #     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
      #     A.ShiftScaleRotate(shift_limit_y=0.1),
      #     A.VerticalFlip(),
      #     A.HorizontalFlip(),
      #     #A.CLAHE(clip_limit=2),
      #     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
      #     A.Sharpen(alpha=1, lightness=0),
      #     A.ColorJitter(saturation=0.2, hue=0.1),
      #     #A.GaussianBlur(blur_limit=(3, 5)),
      #     #A.RandomGamma(), # gamma_limit=(0.8, 1.2)
      #     A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
      #     ToTensorV2()
      # ])
      # self.another_transform4810 = A.Compose([
      #     A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
      #     A.VerticalFlip(),
      #     A.HorizontalFlip(),
      #     #A.CLAHE(clip_limit=2),
      #     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
      #     A.Sharpen(alpha=1, lightness=0),
      #     A.ColorJitter(saturation=0.2, hue=0.1),
      #     #A.GaussianBlur(blur_limit=(3, 5)),
      #     #A.RandomGamma(),
      #     A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
      #     ToTensorV2()
      # ])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
      self.img_path_list = img_path_list
      self.label_list = label_list
      self.transforms = transforms
      self.another_transform1314 = A.Compose([
          A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
          A.VerticalFlip(),
          A.CLAHE(clip_limit=2),
          #A.OneOf([
          #  A.Sharpen(),
          #  A.CLAHE(clip_limit=2),
          #], p=0.5),
          A.RandomBrightnessContrast(),
          A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
          ToTensorV2()
      ])
      self.another_transform137 = A.Compose([
          A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
          A.HorizontalFlip(),
          A.CLAHE(clip_limit=2),
          #A.OneOf([
          #  A.Sharpen(),
          #  A.CLAHE(clip_limit=2),
          #], p=0.5),
          A.RandomBrightnessContrast(),
          A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
          ToTensorV2()
      ])
      self.another_transform810 = A.Compose([
          A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
          #A.OneOf([
          #  A.Sharpen(),
          #  A.CLAHE(clip_limit=2),
          #], p=0.5),
          A.GaussianBlur(blur_limit=(3, 5)),
          A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
          ToTensorV2()
      ]) 
      
    def __getitem__(self, index):
      img_path = self.img_path_list[index]
        
      image = cv2.imread(img_path)
      #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
      if self.label_list is not None:
        label = self.label_list[index]
        
        if self.transforms is not None:
          if label in [13, 14]:
            image = self.another_transform1314(image=image)['image']
          elif label in [1, 3, 7]:
            image = self.another_transform137(image=image)['image']
          elif label in [8, 10]:
            image = self.another_transform810(image=image)['image']
          else:
            image = self.transforms(image=image)['image']
        
        return image, label

      else:
        if self.transforms is not None:
          image = self.transforms(image=image)['image']
          
      return image
        
    def __len__(self):
      return len(self.img_path_list)

In [ ]:
# class CustomDataset(Dataset):
#   def __init__(self, img_path_list, label_list, transforms = None, normalized_mean = None, normalized_std = None):
#     self.img_path_list = img_path_list
#     self.label_list = label_list
#     self.transforms = transforms
#     self.normalized_mean = normalized_mean
#     self.normalized_std = normalized_std

#     # RGB 채널별 평균과 표준편차 초기화
#     self.mean_r, self.mean_g, self.mean_b = [], [], []
#     self.std_r, self.std_g, self.std_b = [], [], []
#     if label_list is not None:
#       self.label_to_images = {label: [] for label in set(self.label_list)}
      
#       for i, label in enumerate(label_list):
#         image_path = img_path_list[i]
#         image = cv2.imread(image_path, cv2.IMREAD_COLOR)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         # 각 라벨에 속하는 이미지를 저장
#         if label not in self.label_to_images:
#           self.label_to_images[label] = []
#         self.label_to_images[label].append(image)
      
#       if (normalized_mean is None) and (normalized_std is None):

#         self.mean_r = np.array([np.mean(self.label_to_images[label][0][:, :, 0]) for label in self.label_to_images])
#         self.mean_g = np.array([np.mean(self.label_to_images[label][0][:, :, 1]) for label in self.label_to_images])
#         self.mean_b = np.array([np.mean(self.label_to_images[label][0][:, :, 2]) for label in self.label_to_images])
#         self.std_r = np.array([np.std(self.label_to_images[label][0][:, :, 0]) for label in self.label_to_images])
#         self.std_g = np.array([np.std(self.label_to_images[label][0][:, :, 1]) for label in self.label_to_images])
#         self.std_b = np.array([np.std(self.label_to_images[label][0][:, :, 2]) for label in self.label_to_images])

#         #print(np.mean(list(self.label_to_images.values())[0][0], axis = (1, 2)).tolist())
#         #print(np.std(list(self.label_to_images.values())[0][0], axis = (1, 2)).tolist())
          
#         #self.all_mean_r, self.all_mean_g, self.all_mean_b = np.mean(list(self.label_to_images.values())[0][0], axis = (0, 1)).tolist()
#         #self.all_std_r, self.all_std_g, self.all_std_b = np.std(list(self.label_to_images.values())[0][0], axis = (0, 1)).tolist()

#         self.all_mean_r = np.mean(list(self.label_to_images.values())[0][0][:, :, 0])
#         self.all_mean_g = np.mean(list(self.label_to_images.values())[0][0][:, :, 1])
#         self.all_mean_b = np.mean(list(self.label_to_images.values())[0][0][:, :, 2])
#         self.all_std_r = np.std(list(self.label_to_images.values())[0][0][:, :, 0])
#         self.all_std_g = np.std(list(self.label_to_images.values())[0][0][:, :, 1])
#         self.all_std_b = np.std(list(self.label_to_images.values())[0][0][:, :, 2])

#         self.all_mean_r /= 255.0
#         self.all_mean_g /= 255.0
#         self.all_mean_b /= 255.0
#         self.all_std_r /= 255.0
#         self.all_std_g /= 255.0
#         self.all_std_b /= 255.0

#         print(self.all_mean_r, self.all_mean_g, self.all_mean_b)
#         print(self.all_std_r, self.all_std_g, self.all_std_b)

#       else:
#         print('normalized_mean {} normalized_std {}'.format(normalized_mean, normalized_std))
#         self.all_mean_r = self.normalized_mean[0]
#         self.all_mean_g = self.normalized_mean[1]
#         self.all_mean_b = self.normalized_mean[2]

#         self.all_std_r = self.normalized_std[0]
#         self.all_std_g = self.normalized_std[1]
#         self.all_std_b = self.normalized_std[2]
      
      
#     else:
#       print('normalized_mean {} normalized_std {}'.format(normalized_mean, normalized_std))
#       self.all_mean_r = self.normalized_mean[0]
#       self.all_mean_g = self.normalized_mean[1]
#       self.all_mean_b = self.normalized_mean[2]

#       self.all_std_r = self.normalized_std[0]
#       self.all_std_g = self.normalized_std[1]
#       self.all_std_b = self.normalized_std[2]


#   def __getitem__(self, index):
#     img_path = self.img_path_list[index]

#     image = cv2.imread(img_path, cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     if self.transforms is not None:
#       image = self.transforms(image = image)['image']
  
#     if self.label_list is not None:
#       label = self.label_list[index]

#       if (self.normalized_mean is None) and (self.normalized_std is None):

#         mean_r, mean_g, mean_b = self.mean_r[label], self.mean_g[label], self.mean_b[label]
#         std_r, std_g, std_b = self.std_r[label], self.std_g[label], self.std_b[label]

#         # RGB 채널별 Normalize
#         image[:, :, 0] = (image[:, :, 0] / 255.0 - mean_r) / std_r
#         image[:, :, 1] = (image[:, :, 1] / 255.0 - mean_g) / std_g
#         image[:, :, 2] = (image[:, :, 2] / 255.0 - mean_b) / std_b

#         return image, label

#       else:

#         mean_r, mean_g, mean_b = self.all_mean_r, self.all_mean_g, self.all_mean_b
#         std_r, std_g, std_b = self.all_std_r, self.all_std_g, self.all_std_b
      
#         # RGB 채널별 Normalize
#         image[:, :, 0] = (image[:, :, 0] / 255.0 - mean_r) / std_r
#         image[:, :, 1] = (image[:, :, 1] / 255.0 - mean_g) / std_g
#         image[:, :, 2] = (image[:, :, 2] / 255.0 - mean_b) / std_b

#         return image, label

#     else:
#       mean_r, mean_g, mean_b = self.all_mean_r, self.all_mean_g, self.all_mean_b
#       std_r, std_g, std_b = self.all_std_r, self.all_std_g, self.all_std_b
      
#       # RGB 채널별 Normalize
#       image[:, :, 0] = (image[:, :, 0] / 255.0 - mean_r) / std_r
#       image[:, :, 1] = (image[:, :, 1] / 255.0 - mean_g) / std_g
#       image[:, :, 2] = (image[:, :, 2] / 255.0 - mean_b) / std_b

#       return image
  
#   def __len__(self):
#     return len(self.img_path_list)

In [ ]:
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.HorizontalFlip(p=0.5),
#                             A.VerticalFlip(p=0.5),
#                             A.RandomRotate90(p=0.5),
#                             A.RandomBrightnessContrast(p=0.5),
#                             A.CLAHE(p=0.5),
#                             A.Normalize(mean=(0.601033, 0.5926225, 0.57677054), std=(0.08033815, 0.084218614, 0.088231266), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])

# A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                       A.HorizontalFlip(),
#                       A.VerticalFlip(),
#                       A.Rotate(limit=45),
#                       #A.RandomRotate90(),
#                       # A.OneOf([
#                       #   A.CLAHE(clip_limit=2),
#                       #   A.RandomBrightnessContrast(),
#                       # ], p=0.5),
#                       A.CLAHE(clip_limit=2),
#                       A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
#                       #A.Sharpen(alpha=1, lightness=0),
#                       #A.ColorJitter(saturation=0.2, hue=0.1),
#                       A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
#                       ToTensorV2()

train_transform = A.Compose([
                      A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                      A.HorizontalFlip(),
                      A.VerticalFlip(),
                      A.Rotate(limit=45),
                      A.CLAHE(clip_limit=2),
                      #A.OneOf([
                      #  A.Sharpen(),
                      #  A.CLAHE(clip_limit=2),
                      #], p=0.5),
                      A.RandomBrightnessContrast(),
                      A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
                      ToTensorV2()
])

test_transform = A.Compose([
                      A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                      A.Normalize(mean=(meanR, meanG, meanB), std=(stdR, stdG, stdB), max_pixel_value=255.0, always_apply=False, p=1.0),
                      ToTensorV2()
])

In [ ]:
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#                             ToTensorV2(),
#                             ])

# test_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#                             ToTensorV2(),
#                             ])

In [ ]:

# a_list = train['label'].value_counts()
# class_counts = train['label'].value_counts().to_list()
# class_weights = [1 / a_list[i] for i in range(len(class_counts))]
# num_samples = sum(class_counts)
# labels_lists = train['label'].to_list()
# weights = [class_weights[labels_lists[i]] for i in range(int(num_samples))]
# sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)

In [ ]:
# train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
# train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler=sampler, shuffle=False, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform, normalized_mean = (0.485, 0.456, 0.406), normalized_std = (0.229, 0.224, 0.225))
# train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle = False, num_workers = 0)

# val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform, normalized_mean = (0.485, 0.456, 0.406), normalized_std = (0.229, 0.224, 0.225))
# val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle = False, num_workers = 0)

# **MODELING**

### TO DO
- 전이학습 (transfer learning)

In [ ]:
!pip -q install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00


In [ ]:
import timm

In [ ]:
!pip install --upgrade efficientnet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=cc7435fcaa650cbb58b4b87e82e25f2be40f69bfabbecfda00142b2638e248fd
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
# from torchvision.models import inception_v3 as Inception_v3
# from torchvision.models import efficientnet_b3 as efficientNet_b3

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(uniques)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True) # 224 x 224
        #self.fc = nn.Linear(1000, 500)
        self.classifier = nn.Linear(1000, num_classes)

        #self.backbone = EfficientNet.from_pretrained('efficientnet-b0', in_channels=3, num_classes=num_classes, dropout_rate=0.3)

        #self.backbone = EfficientNet.from_pretrained('efficientnet-b3', in_channels=3, num_classes=num_classes)

        #self.backbone = timm.create_model('efficientnet_b3', pretrained=True, num_classes=num_classes) # 300 x 300
        
        #self.backbone = timm.create_model('tf_efficientnet_b7_ns', pretrained = True, num_classes = num_classes) # 600 x 600
        
    def forward(self, x):
        x = self.backbone(x)
        #x = self.fc(x)
        x = self.classifier(x)
        return x

In [ ]:
from collections import Counter

def calculate_alpha(labels):
    # Step 1: Calculate class frequencies
    class_frequencies = Counter(labels)
    num_classes = len(class_frequencies)

    # Step 2: Calculate inverse class weights
    total_count = len(labels)
    class_weights = torch.tensor([total_count / class_frequencies[label] for label in range(num_classes)])

    # Step 3: Normalize class weights to [0, 1]
    alpha = class_weights / class_weights.sum()

    return alpha.tolist() * 10

# weights = calculate_alpha(labels)

In [ ]:
import math
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [ ]:
class RAdamW(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))

        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdamW, self).__setstate__(state)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (
                                    1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.mul_(1 - group['lr'] * group['weight_decay'])
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(exp_avg, denom, value=-step_size * group['lr'])
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.mul_(1 - group['lr'] * group['weight_decay'])
                    p_data_fp32.add_(exp_avg, alpha=-step_size * group['lr'])
                    p.data.copy_(p_data_fp32)

        return loss

In [ ]:
from collections import defaultdict


class Lookahead(Optimizer):
    r'''Implements Lookahead optimizer.

    It's been proposed in paper: Lookahead Optimizer: k steps forward, 1 step back
    (https://arxiv.org/pdf/1907.08610.pdf)

    Args:
        optimizer: The optimizer object used in inner loop for fast weight updates.
        alpha:     The learning rate for slow weight update.
                   Default: 0.5
        k:         Number of iterations of fast weights updates before updating slow
                   weights.
                   Default: 5

    Example:
        > optim = Lookahead(optimizer)
        > optim = Lookahead(optimizer, alpha=0.6, k=10)
    '''

    def __init__(self, optimizer, alpha=0.5, k=5):
        assert (0.0 <= alpha <= 1.0)
        assert (k >= 1)
        self.optimizer = optimizer
        self.defaults = optimizer.defaults
        self.alpha = alpha
        self.k = k
        self.k_counter = 0
        self.param_groups = self.optimizer.param_groups
        self.state = defaultdict(dict)
        self.slow_weights = [[param.clone().detach() for param in group['params']] for group in self.param_groups]

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        self.k_counter += 1
        if self.k_counter >= self.k:
            for group, slow_weight in zip(self.param_groups, self.slow_weights):
                for param, weight in zip(group['params'], slow_weight):
                    #weight.data.add_(self.alpha, (param.data - weight.data))
                    weight.data.add_(other=param.data - weight.data, alpha=self.alpha)

                    param.data.copy_(weight.data)
            self.k_counter = 0
        return loss

    def __getstate__(self):
        return {
            'state': self.state,
            'optimizer': self.optimizer,
            'alpha': self.alpha,
            'k': self.k,
            'k_counter': self.k_counter
        }

    def state_dict(self):
        return self.optimizer.state_dict()

    def load_state_dict(self, state_dict):
        self.optimizer.load_state_dict(state_dict)

# from collections import defaultdict
# from itertools import chain
# from torch.optim import Optimizer

# class Lookahead(Optimizer):
#     def __init__(self, optimizer, k=5, alpha=0.5):
#         self.optimizer = optimizer
#         self.k = k
#         self.alpha = alpha
#         self.param_groups = self.optimizer.param_groups
#         self.state = defaultdict(dict)
#         self.fast_state = self.optimizer.state
#         for group in self.param_groups:
#             group["counter"] = 0
    
#     def update(self, group):
#         for fast in group["params"]:
#             param_state = self.state[fast]
#             if "slow_param" not in param_state:
#                 param_state["slow_param"] = torch.zeros_like(fast.data)
#                 param_state["slow_param"].copy_(fast.data)
#             slow = param_state["slow_param"]
#             slow += (fast.data - slow) * self.alpha
#             fast.data.copy_(slow)
    
#     def update_lookahead(self):
#         for group in self.param_groups:
#             self.update(group)

#     def step(self, closure=None):
#         loss = self.optimizer.step(closure)
#         for group in self.param_groups:
#             if group["counter"] == 0:
#                 self.update(group)
#             group["counter"] += 1
#             if group["counter"] >= self.k:
#                 group["counter"] = 0
#         return loss

#     def state_dict(self):
#         fast_state_dict = self.optimizer.state_dict()
#         slow_state = {
#             (id(k) if isinstance(k, torch.Tensor) else k): v
#             for k, v in self.state.items()
#         }
#         fast_state = fast_state_dict["state"]
#         param_groups = fast_state_dict["param_groups"]
#         return {
#             "fast_state": fast_state,
#             "slow_state": slow_state,
#             "param_groups": param_groups,
#         }

#     def load_state_dict(self, state_dict):
#         slow_state_dict = {
#             "state": state_dict["slow_state"],
#             "param_groups": state_dict["param_groups"],
#         }
#         fast_state_dict = {
#             "state": state_dict["fast_state"],
#             "param_groups": state_dict["param_groups"],
#         }
#         super(Lookahead, self).load_state_dict(slow_state_dict)
#         self.optimizer.load_state_dict(fast_state_dict)
#         self.fast_state = self.optimizer.state

#     def add_param_group(self, param_group):
#         param_group["counter"] = 0
#         self.optimizer.add_param_group(param_group)

In [ ]:
from torch.optim import Optimizer

class Lion(Optimizer):
    def __init__( self, params, lr: float = 1e-4, betas: tuple= (0.9, 0.99), weight_decay: float = 0.0):
        
        assert lr > 0.
        assert all([0. <= beta <= 1. for beta in betas])
        defaults = dict(lr = lr, betas = betas, weight_decay = weight_decay)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step( self, closure= None):

        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in filter(lambda p: p.grad is not None, group['params']):
                #parameter
                grad, lr, wd, beta1, beta2, state = p.grad, group['lr'], group['weight_decay'], *group['betas'], self.state[p]
                if len(state) == 0: state['exp_avg'] = torch.zeros_like(p)
                exp_avg = state['exp_avg']
                
                 # Lion optimizer
                p.data.mul_(1 - lr * wd)
                update = exp_avg.clone().lerp_(grad, 1 - beta1)
                p.add_(torch.sign(update), alpha = -lr)
                exp_avg.lerp_(grad, 1 - beta2)
        return loss

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
      self.patience = patience
      self.verbose = verbose
      self.counter = 0
      self.best_score = None
      self.early_stop = False
      self.val_loss_min = np.Inf
      self.delta = delta

    def __call__(self, score):
      if self.best_score is None:
        self.best_score = score
      elif score < self.best_score + self.delta:
        self.counter += 1
        print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
        if self.counter >= self.patience:
          self.early_stop = True
      else:
        self.best_score = score
        self.counter = 0
        
      return self.early_stop

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class CrossEntropyLossWeight(nn.Module):
    def __init__(self, weight=None, reduction='mean'):
      super(CrossEntropyLossWeight, self).__init__()
      self.reduction = reduction
      self.weight = weight

    def forward(self, inputs, targets):
      #ce_loss = nn.CrossEntropyLoss(self.weight)
      ce_loss = F.cross_entropy(inputs, targets, reduction=self.reduction, weight=self.weight)
      return ce_loss

class FocalLossClassWeight(nn.Module):
    def __init__(self, gamma=2, weight=None, reduction='mean'):
        super(FocalLossClassWeight, self).__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.weight = weight

    def forward(self, inputs, targets):
        #ce_loss = nn.CrossEntropyLoss(self.weight)
        ce_loss = F.cross_entropy(inputs, targets, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

# class FocalLoss(nn.Module):
#     def __init__(self, gamma=0, alpha=None, size_average=True):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.alpha = alpha
#         if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
#         self.alpha[18] = 1-alpha
#         if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
#         self.size_average = size_average

#     def forward(self, input, target):
#         if input.dim()>2:
#             input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
#             input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
#             input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
#         target = target.view(-1,1)

#         logpt = F.log_softmax(input)
#         logpt = logpt.gather(1,target)
#         logpt = logpt.view(-1)
#         pt = Variable(logpt.data.exp())

#         if self.alpha is not None:
#             if self.alpha.type()!=input.data.type():
#                 self.alpha = self.alpha.type_as(input.data)
#             at = self.alpha.gather(0,target.data.view(-1))
#             logpt = logpt * at

#         loss = -1 * (1-pt)**self.gamma * logpt
#         if self.size_average: return loss.mean()
#         else: return loss.sum()

class FocalLoss(nn.Module):
    "Focal loss implemented using F.cross_entropy"
    def __init__(self, gamma: float = 2.0, weight=None, reduction: str = 'mean') -> None:
        super().__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, inp: torch.Tensor, targ: torch.Tensor):
        ce_loss = F.cross_entropy(inp, targ, weight=self.weight, reduction="none")
        p_t = torch.exp(-ce_loss)
        loss = (1 - p_t)**self.gamma * ce_loss
        if self.reduction == "mean":
            loss = loss.mean()
        elif self.reduction == "sum":
            loss = loss.sum()
        return loss

class FocalLoss0(nn.Module):
    """
    binary focal loss
    """
    def __init__(self, alpha=0.25, gamma=2, weight=None):
        super(FocalLoss0, self).__init__()
        #self.weight = torch.Tensor([alpha, 1-alpha])
        self.weight = weight
        self.nllLoss = nn.NLLLoss(weight=self.weight)
        self.gamma = gamma

    def forward(self, input, target):
        softmax = F.softmax(input, dim=1)
        log_logits = torch.log(softmax)
        fix_weights = (1 - softmax) ** self.gamma
        logits = fix_weights * log_logits
        return self.nllLoss(logits, target)

# 0.55 0.64
class FocalLoss1(nn.Module):
    """Implementation of Facal Loss"""
    def __init__(self, gamma=2, weight=None, reduction="mean"):
        super(FocalLoss1, self).__init__()
        self.weighted_cs = nn.CrossEntropyLoss(weight=weight, reduction="none")
        self.cs = nn.CrossEntropyLoss(reduction="none")
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, predicted, target):
        """
        predicted: [batch_size, n_classes]
        target: [batch_size]
        """
        pt = 1/torch.exp(self.cs(predicted,target))
        entropy_loss = self.weighted_cs(predicted, target)
        focal_loss = ((1-pt)**self.gamma)*entropy_loss
        if self.reduction =="none":
            return focal_loss
        elif self.reduction == "mean":
            return focal_loss.mean()

class FocalLoss2(nn.Module):
    
    def __init__(self, 
                 gamma=2., weight=None, reduction='none'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob, 
            target_tensor, 
            weight=self.weight,
            reduction = self.reduction
        )

class FocalLossWeighted(nn.Module):
    def __init__(self, gamma=2, reduction='mean'):
        super(FocalLossWeighted, self).__init__()
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets, weights=None):

        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        targets = targets.view(-1)

        if weights is not None:
            weights = torch.tensor(weights)
            focal_weight = weights[torch.tensor(targets)]
            focal_weight = focal_weight ** self.gamma
            focal_loss = -1 * focal_weight * ce_loss
        else:
            focal_loss = -1 * (1 - torch.exp(-ce_loss)) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            focal_loss = torch.mean(focal_loss)
        elif self.reduction == 'sum':
            focal_loss = torch.sum(focal_loss)

        return focal_loss

class InverseClassFreqFocalLoss(nn.Module):
    def __init__(self, gamma=0, reduction='mean'):
        super(InverseClassFreqFocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets, weights=None):
        targets = targets.view(-1, 1)
        logpt = F.log_softmax(inputs, dim=1)
        logpt = logpt.gather(1, targets)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if weights is not None:
          weights = torch.tensor(weights)
          focal_weight = weights[torch.tensor(targets)]
          logpt = logpt * focal_weight
          focal_loss = -((1 - pt) ** self.gamma) * logpt
        else:
          focal_loss = -((1 - pt) ** self.gamma) * logpt

        if self.reduction == 'mean':
            focal_loss = torch.mean(focal_loss)
        elif self.reduction == 'sum':
            focal_loss = torch.sum(focal_loss)
        
        return focal_loss

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_weights = compute_class_weight(class_weight = "balanced", classes=np.unique(train_df['label']), y=train_df['label'])
#class_weights = torch.from_numpy(np.array(class_weights))
class_weights = torch.FloatTensor(class_weights)
class_weights

tensor([15.1623,  0.5927,  1.2548,  0.8664, 12.9962,  3.3694,  1.8379,  1.3996,
        60.6491,  3.1921,  0.3058,  1.2813,  8.2703, 10.7028,  6.7388,  1.1231,
        36.3895,  3.5676,  0.1295])

In [ ]:
weights_dict = {label: count for label, count in zip(sorted(train_df.label.unique()), train_df.label.value_counts().sort_index(ascending = True) / len(train_df))}
weights = list(weights_dict.values())

In [ ]:
sub_weights = 1 - np.array(weights)
inverse_weights = 1 / np.array(weights)

In [ ]:
def add_weight_decay(model, weight_decay=1e-4, skip_list=()):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue  # frozen weights
        if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list:
            no_decay.append(param)
        else:
            decay.append(param)
    return [
        {'params': no_decay, 'weight_decay': 0.},
        {'params': decay, 'weight_decay': weight_decay}]

In [ ]:
def plot_confusion_matrix(cm, classes, runid, epoch, f1, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
  plt.title(f'{title}-{runid}-{epoch}-{f1:.4f}')
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

  thresh = cm.max() / 2
  for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.savefig(f'./cm-{runid}.jpg', dpi = 400)
  plt.clf()

In [ ]:
def trains(model, optimizer, train_loader, val_loader, scheduler, criterion, device, fold=None):
    model.to(device)

    #class_weights = compute_class_weight(class_weight = "balanced", classes=train_df.label.unique(), y=train_df['label'])
    #class_weights = torch.FloatTensor(class_weights)

    # 성능O
    #weights_dict = {label: count for label, count in zip(sorted(train_df.label.unique()), train_df.label.value_counts().sort_index(ascending = True) / len(train_df))}
    #weights = list(weights_dict.values())
    #sub_weights = 1 - np.array(weights)
    #inverse_weights = 1 / np.array(weights)
    #class_weights = torch.FloatTensor(sub_weights)

    #criterion = nn.CrossEntropyLoss().to(device)
    
    #criterion = FocalLoss(gamma=2, alpha=0.25)
    
    #criterion = FocalLoss()

    #criterion = FocalLoss0(alpha=0.25, gamma=2, weight=class_weights)
    #criterion = FocalLoss1(gamma=2, weight=class_weights, reduction='mean')
    #criterion = FocalLoss2(gamma=2, weight=class_weights, reduction='mean')
    
    #criterion = CrossEntropyLossWeight(weight=class_weights, reduction='mean')
    #criterion = FocalLossClassWeight(gamma=2, weight=class_weights, reduction='mean')
    #criterion = FocalLossWeighted(gamma=2, reduction='mean')
    #criterion = InverseClassFreqFocalLoss(gamma=2, reduction='mean')
    
    best_score = 0
    best_model = None
    fold_scores = {}
    early_stopping = EarlyStopping(patience=5, verbose=True)
    scaler = GradScaler()

    print(f'----- fold: {fold} -----')
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)
            #labels = torch.max(labels, 1)

            optimizer.zero_grad()
            
            with autocast():
              output = model(imgs)
              
              loss = criterion(output, labels)
          
            # FocalLoss nessesary
            #loss = criterion(output, labels, weights=sub_weights)

            #loss.backward()
            scaler.scale(loss).backward()

            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()
            
            #train_loss.append(loss.item())
            train_loss.append(loss.item() * imgs.size(0))
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device, fold=fold)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model

        stop = early_stopping(_val_score)
        if stop:

          break
    
    # 모델 가중치 저장
    PATH = f"./fold{fold}_best_model_weights.pt"
    torch.save(best_model.state_dict(), PATH)

    print('Fold {} Best val_score: {:.5f}'.format(fold, best_score))
    fold_scores[fold] = best_score

    return best_model

In [ ]:
# def validation(model, criterion, val_loader, device, fold=None):
#     model.eval()
#     val_loss = []
#     preds, true_labels = [], []

#     with torch.no_grad():
#         for imgs, labels in tqdm(iter(val_loader)):
#             imgs = imgs.float().to(device)
#             labels = labels.type(torch.LongTensor).to(device)
            
#             pred = model(imgs)
            
#             loss = criterion(pred, labels)
            
#             preds += pred.argmax(1).detach().cpu().numpy().tolist()
#             true_labels += labels.detach().cpu().numpy().tolist()
            
#             val_loss.append(loss.item())
        
#         _val_loss = np.mean(val_loss)
#         _val_score = f1_score(true_labels, preds, average='weighted')
    
#     return _val_loss, _val_score

def validation(model, criterion, val_loader, device, fold=None):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    for imgs, labels in tqdm(iter(val_loader)):
      imgs = imgs.float().to(device)
      labels = labels.type(torch.LongTensor).to(device)
      #labels = torch.max(labels, 1)
            
      with torch.no_grad():
        pred = model(imgs)
            
        loss = criterion(pred, labels)
            
      preds += pred.argmax(1).detach().cpu().numpy().tolist()
      true_labels += labels.detach().cpu().numpy().tolist()
            
      #val_loss.append(loss.item())
      val_loss.append(loss.item() * imgs.size(0))
        
    _val_loss = np.mean(val_loss)
    _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
def inference(model, test_loader, device, fold=None):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    #preds = le.inverse_transform(preds)
    return preds

# GPU 학습! 
병렬처리
- torch.nn.DataParallel() **VS** torch.nn.parallel.DistributedDataParallel(개선)
- num_workers
- .cuda()
- pin_memory=True
- DistributedSampler

-> 병렬처리를 함으로써 메모리 초과 문제를 해결
  - batch_size = 128
  - efficientnet_b3
  - efficientnet_b7
  - tf_efficientnet_b7_ns
  - Epochs 늘리기
  - KFold수 늘리기

# TO DO!
 - efficientnet_b3 **메모리 초과 문제로 x**
 - tf_efficientnet_b7_ns **메모리 초과 문제로 x**
 - FocalLoss 동일한 가중치 **성능 O**
 - criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
  + FocalLoss class_weights
 - FocalLoss의 가중치를 예측을 잘 못하는 클래스에는 높은 가중치를 주는 방식으로 성능에 따른 가중치 방식을 사용 (역 클래스 빈도) **성능 O**
 - Confusion matrix (모델이 예측을 잘하는지 분석)
 - RAadam **성능X**
 - RAdamW **성능X**
 - Lookahead **성능X**
 - StepLR **성능X**
 - OneCycleLR **성능X**
 - GradScaler **성능O**

- Baseline (Adam + FocalLoss + CosineAnnealingLR)

Epoch [1], Train Loss : [0.29706] Val Loss : [0.38998] Val Weighted F1 Score : [0.61369]
100%
44/44 [12:39<00:00, 12.63s/it]
100%
11/11 [00:55<00:00, 4.95s/it]
Epoch [2], Train Loss : [0.11559] Val Loss : [0.23825] Val Weighted F1 Score : [0.65413]

- Baseline + FClayer 추가

Epoch [1], Train Loss : [0.27609] Val Loss : [0.38474] Val Weighted F1 Score : [0.60448]
100%
87/87 [12:11<00:00, 6.93s/it]
100%
22/22 [00:51<00:00, 2.04s/it]
Epoch [2], Train Loss : [0.12258] Val Loss : [0.38993] Val Weighted F1 Score : [0.63627]

- FocalLoss

Epoch [1], Train Loss : [0.87108] Val Loss : [1.04524] Val Weighted F1 Score : [0.56942]
100%
87/87 [11:41<00:00, 6.65s/it]
100%
22/22 [00:54<00:00, 2.29s/it]
Epoch [2], Train Loss : [0.31641] Val Loss : [0.81740] Val Weighted F1 Score : [0.61748]

- Adam + GradScaler

Epoch [1], Train Loss : [0.27282] Val Loss : [0.39775] Val Weighted F1 Score : [0.61214]
100%
87/87 [12:44<00:00, 6.53s/it]
100%
22/22 [00:50<00:00, 2.12s/it]
Epoch [2], Train Loss : [0.11217] Val Loss : [0.25941] Val Weighted F1 Score : [0.67919]

- AdamW + GradScaler

Epoch [1], Train Loss : [0.27343] Val Loss : [0.34507] Val Weighted F1 Score : [0.60251]
100%
87/87 [16:13<00:00, 8.84s/it]
100%
22/22 [00:52<00:00, 2.20s/it]
Epoch [2], Train Loss : [0.11938] Val Loss : [0.24513] Val Weighted F1 Score : [0.67024]
2%

- Adam + GradScaler + add_weight_decay

Epoch [1], Train Loss : [0.27676] Val Loss : [0.37816] Val Weighted F1 Score : [0.63718]
100%
87/87 [14:41<00:00, 7.80s/it]
100%
22/22 [00:51<00:00, 2.07s/it]
Epoch [2], Train Loss : [0.11702] Val Loss : [0.27282] Val Weighted F1 Score : [0.68025]


Epoch [1], Train Loss : [26.99901] Val Loss : [24.46869] Val Weighted F1 Score : [0.68090]
100%
87/87 [11:50<00:00, 6.39s/it]
100%
22/22 [00:49<00:00, 2.13s/it]
Epoch [2], Train Loss : [10.23576] Val Loss : [19.18536] Val Weighted F1 Score : [0.71826]

In [ ]:
##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Created by: Hang Zhang
## ECE Department, Rutgers University
## Email: zhang.hang@rutgers.edu
## Copyright (c) 2017
##
## This source code is licensed under the MIT-style license found in the
## LICENSE file in the root directory of this source tree
##+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

"""Encoding Data Parallel"""
import threading
import functools
import torch
from torch.autograd import Variable, Function
import torch.cuda.comm as comm
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn.parallel.parallel_apply import get_a_var
from torch.nn.parallel._functions import ReduceAddCoalesced, Broadcast

torch_ver = torch.__version__[:3]

__all__ = ['allreduce', 'DataParallelModel', 'DataParallelCriterion',
           'patch_replication_callback']

def allreduce(*inputs):
    """Cross GPU all reduce autograd operation for calculate mean and
    variance in SyncBN.
    """
    return AllReduce.apply(*inputs)

class AllReduce(Function):
    @staticmethod
    def forward(ctx, num_inputs, *inputs):
        ctx.num_inputs = num_inputs
        ctx.target_gpus = [inputs[i].get_device() for i in range(0, len(inputs), num_inputs)]
        inputs = [inputs[i:i + num_inputs]
                 for i in range(0, len(inputs), num_inputs)]
        # sort before reduce sum
        inputs = sorted(inputs, key=lambda i: i[0].get_device())
        results = comm.reduce_add_coalesced(inputs, ctx.target_gpus[0])
        outputs = comm.broadcast_coalesced(results, ctx.target_gpus)
        return tuple([t for tensors in outputs for t in tensors])

    @staticmethod
    def backward(ctx, *inputs):
        inputs = [i.data for i in inputs]
        inputs = [inputs[i:i + ctx.num_inputs]
                 for i in range(0, len(inputs), ctx.num_inputs)]
        results = comm.reduce_add_coalesced(inputs, ctx.target_gpus[0])
        outputs = comm.broadcast_coalesced(results, ctx.target_gpus)
        return (None,) + tuple([Variable(t) for tensors in outputs for t in tensors])

class Reduce(Function):
    @staticmethod
    def forward(ctx, *inputs):
        ctx.target_gpus = [inputs[i].get_device() for i in range(len(inputs))]
        inputs = sorted(inputs, key=lambda i: i.get_device())
        return comm.reduce_add(inputs)

    @staticmethod
    def backward(ctx, gradOutput):
        return Broadcast.apply(ctx.target_gpus, gradOutput)


class DataParallelModel(DataParallel):
    """Implements data parallelism at the module level.
    This container parallelizes the application of the given module by
    splitting the input across the specified devices by chunking in the
    batch dimension.
    In the forward pass, the module is replicated on each device,
    and each replica handles a portion of the input. During the backwards pass, gradients from each replica are summed into the original module.
    Note that the outputs are not gathered, please use compatible
    :class:`encoding.parallel.DataParallelCriterion`.
    The batch size should be larger than the number of GPUs used. It should
    also be an integer multiple of the number of GPUs so that each chunk is
    the same size (so that each GPU processes the same number of samples).
    Args:
        module: module to be parallelized
        device_ids: CUDA devices (default: all devices)
    Reference:
        Hang Zhang, Kristin Dana, Jianping Shi, Zhongyue Zhang, Xiaogang Wang, Ambrish Tyagi,
        Amit Agrawal. “Context Encoding for Semantic Segmentation.
        *The IEEE Conference on Computer Vision and Pattern Recognition (CVPR) 2018*
    Example::
        >>> net = encoding.nn.DataParallelModel(model, device_ids=[0, 1, 2])
        >>> y = net(x)
    """
    def gather(self, outputs, output_device):
        return outputs

    def replicate(self, module, device_ids):
        modules = super(DataParallelModel, self).replicate(module, device_ids)
        return modules


class DataParallelCriterion(DataParallel):
    """
    Calculate loss in multiple-GPUs, which balance the memory usage for
    Semantic Segmentation.
    The targets are splitted across the specified devices by chunking in
    the batch dimension. Please use together with :class:`encoding.parallel.DataParallelModel`.
    Reference:
        Hang Zhang, Kristin Dana, Jianping Shi, Zhongyue Zhang, Xiaogang Wang, Ambrish Tyagi,
        Amit Agrawal. “Context Encoding for Semantic Segmentation.
        *The IEEE Conference on Computer Vision and Pattern Recognition (CVPR) 2018*
    Example::
        >>> net = encoding.nn.DataParallelModel(model, device_ids=[0, 1, 2])
        >>> criterion = encoding.nn.DataParallelCriterion(criterion, device_ids=[0, 1, 2])
        >>> y = net(x)
        >>> loss = criterion(y, target)
    """
    def forward(self, inputs, *targets, **kwargs):
        # input should be already scatterd
        # scattering the targets instead
        if not self.device_ids:
            return self.module(inputs, *targets, **kwargs)
        targets, kwargs = self.scatter(targets, kwargs, self.device_ids)
        if len(self.device_ids) == 1:
            return self.module(inputs, *targets[0], **kwargs[0])
        replicas = self.replicate(self.module, self.device_ids[:len(inputs)])
        outputs = _criterion_parallel_apply(replicas, inputs, targets, kwargs)
        return Reduce.apply(*outputs) / len(outputs)


def _criterion_parallel_apply(modules, inputs, targets, kwargs_tup=None, devices=None):
    assert len(modules) == len(inputs)
    assert len(targets) == len(inputs)
    if kwargs_tup:
        assert len(modules) == len(kwargs_tup)
    else:
        kwargs_tup = ({},) * len(modules)
    if devices is not None:
        assert len(modules) == len(devices)
    else:
        devices = [None] * len(modules)

    lock = threading.Lock()
    results = {}
    if torch_ver != "0.3":
        grad_enabled = torch.is_grad_enabled()

    def _worker(i, module, input, target, kwargs, device=None):
        if torch_ver != "0.3":
            torch.set_grad_enabled(grad_enabled)
        if device is None:
            device = get_a_var(input).get_device()
        try:
            with torch.cuda.device(device):
                output = module(*(input + target), **kwargs)
            with lock:
                results[i] = output
        except Exception as e:
            with lock:
                results[i] = e

    if len(modules) > 1:
        threads = [threading.Thread(target=_worker,
                                    args=(i, module, input, target,
                                          kwargs, device),)
                   for i, (module, input, target, kwargs, device) in
                   enumerate(zip(modules, inputs, targets, kwargs_tup, devices))]

        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
    else:
        _worker(0, modules[0], inputs[0], kwargs_tup[0], devices[0])

    outputs = []
    for i in range(len(inputs)):
        output = results[i]
        if isinstance(output, Exception):
            raise output
        outputs.append(output)
    return outputs

In [ ]:
skf = sklearn.model_selection.StratifiedKFold(n_splits=10, shuffle=False)
t = train_df.label
fold_preds = []
for fold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(t)), t)):
  data_train = train_df.loc[train_index]
  data_validation = train_df.loc[val_index]

  a_list = data_train['label'].value_counts()
  class_counts = data_train['label'].value_counts().to_list()
  class_weights = [1 / a_list[i] for i in range(len(class_counts))]
  num_samples = sum(class_counts)
  labels_lists = data_train['label'].to_list()
  classweights = [class_weights[labels_lists[i]] for i in range(int(num_samples))]
  sampler = WeightedRandomSampler(torch.DoubleTensor(classweights), int(num_samples), replacement=True)

  train_dataset = CustomDataset(data_train['img_path'].values, data_train['label'].values, train_transform)
  train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler=sampler, shuffle=False, num_workers=2, pin_memory=True)
  
  val_dataset = CustomDataset(data_validation['img_path'].values, data_validation['label'].values, test_transform)
  val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2, pin_memory=True) # num_workers=4, pin_memory=True

  test_df = pd.read_csv(dataset_dir + '/test.csv')
  test_df['img_path'] = dataset_dir + test_df['img_path'].str.split('/').str[1] + '/' + test_df['img_path'].str.split('/').str[2]
  test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
  test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2, pin_memory=True) # num_workers=4, pin_memory=True

  model = BaseModel()
  #model = DataParallelModel(model)
  criterion = FocalLoss()
  #criterion = DataParallelCriterion(criterion)

  #model = nn.DataParallel(model, device_ids=[0, 1, 2, 3], output_device=1) #, output_device=0
  
  #weight_decay = CFG['LEARNING_RATE'] # 3e-4, 1e-4
  #parameters = add_weight_decay(model, weight_decay)
  optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE']) # betas=(0.9, 0.999)
  #optimizer = Lion(parameters, lr = CFG['LEARNING_RATE'], betas = (0.9, 0.99)) 
  
  #optimizer = torch.optim.Adam(model.parameters(), lr = CFG["LEARNING_RATE"], betas=(0.9, 0.999))
  #optimizer = torch.optim.AdamW(model.parameters(), lr = CFG["LEARNING_RATE"])
  #optimizer = RAdam(model.parameters(), lr=CFG['LEARNING_RATE'], betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=False)
  #optimizer = RAdamW(model.parameters(), lr=CFG['LEARNING_RATE'], betas=(0.9, 0.999), weight_decay=0.2) 
  #optimizer = Lookahead(optimizer, k=5, alpha=0.25)
  
  scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)
  #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
  #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CFG['LEARNING_RATE'], epochs=20, steps_per_epoch=len(train_loader), pct_start=0.03)
  #scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=1e-6)
  
  infer_model = trains(model, optimizer, train_loader, val_loader, scheduler, criterion, device, fold=fold)

  preds = inference(infer_model, test_loader, device, fold=fold)
  sample_submission = pd.read_csv(dataset_dir + 'sample_submission.csv')
  sample_submission['label'] = preds
  sample_submission['label'] = sample_submission['label'].map(inverse_dict)
  fold_preds.append(sample_submission['label'].values)
  os.makedirs('./output/', exist_ok=True)
  sample_submission.to_csv(f'./output/fold{fold}.csv', index=False)

----- fold: 0 -----


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [1], Train Loss : [28.49979] Val Loss : [30.13193] Val Weighted F1 Score : [0.60140]


  0%|          | 0/87 [00:00<?, ?it/s]

...

Epoch [13], Train Loss : [2.28515] Val Loss : [15.94160] Val Weighted F1 Score : [0.76482]
EarlyStopping counter: 4 out of 5


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch [14], Train Loss : [2.26112] Val Loss : [15.23752] Val Weighted F1 Score : [0.77414]
EarlyStopping counter: 5 out of 5
Fold 4 Best val_score: 0.77722


  0%|          | 0/25 [00:00<?, ?it/s]

batch size 32
kfold 5
-

In [ ]:
fold_preds

[array(['몰딩수정', '오염', '훼손', '몰딩수정', '오염', '훼손', '오염', '훼손', '몰딩수정', '오타공',
        '몰딩수정', '오타공', '오염', '오염', '걸레받이수정', '오염', '몰딩수정', '걸레받이수정', '오염',
        '오염', '오염', '훼손', '오염', '석고수정', '오염', '오염', '훼손', '오염', '걸레받이수정',
        '오염', '오염', '훼손', '훼손', '오염', '곰팡이', '훼손', '오염', '훼손', '훼손', '훼손',
        '오염', '오염', '석고수정', '면불량', '면불량', '석고수정', '오염', '오염', '오염', '오염',
        '오염', '오염', '오염', '석고수정', '훼손', '훼손', '훼손', '오염', '면불량', '오염',
        '오타공', '몰딩수정', '오염', '오염', '훼손', '오염', '오염', '오염', '오염', '석고수정',
        '오염', '이음부불량', '오염', '오염', '훼손', '석고수정', '훼손', '울음', '오염', '오염',
        '면불량', '석고수정', '몰딩수정', '면불량', '걸레받이수정', '훼손', '들뜸', '오염', '오염',
        '걸레받이수정', '오염', '훼손', '훼손', '오염', '면불량', '훼손', '터짐', '오타공', '석고수정',
        '오염', '피스', '몰딩수정', '오염', '터짐', '터짐', '오염', '가구수정', '터짐', '훼손',
        '면불량', '터짐', '훼손', '훼손', '오염', '오염', '석고수정', '오염', '훼손', '녹오염',
        '훼손', '면불량', '훼손', '훼손', '석고수정', '훼손', '훼손', '몰딩수정', '훼손', '오염',
        '오타공', '훼손', '몰딩수정', '꼬임', '오염', '몰딩수

In [ ]:
# s1 = pd.read_csv('/content/fold0.csv')
# s2 = pd.read_csv('/content/fold1.csv')
# s3 = pd.read_csv('/content/fold2.csv')
# s4 = pd.read_csv('/content/fold3.csv')
# s5 = pd.read_csv('/content/fold4.csv')
#fold_preds = np.stack([s1['label'].values, s2['label'].values, s3['label'].values, s4['label'].values, s5['label'].values], axis = 0)

In [ ]:
out = []

cols = list(zip(*fold_preds))
for c in cols:
  most = collections.Counter(c).most_common()[0][0]
  out.append(most)

sample_submission = pd.read_csv(dataset_dir + 'sample_submission.csv')
sample_submission['label'] = out
sample_submission.to_csv('fold_voting.csv', index=False)

In [ ]:
sample_submission

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,꼬임
788,TEST_788,터짐
789,TEST_789,오염
790,TEST_790,훼손


# **끝**

In [ ]:
test_df = pd.read_csv(dataset_dir + '/test.csv')
test_df['img_path'] = dataset_dir + test_df['img_path'].str.split('/').str[1] + '/' + test_df['img_path'].str.split('/').str[2]
test_df

,id,img_path
0,TEST_000,/content/drive/MyDrive/open (3)/test/000.png
1,TEST_001,/content/drive/MyDrive/open (3)/test/001.png
2,TEST_002,/content/drive/MyDrive/open (3)/test/002.png
3,TEST_003,/content/drive/MyDrive/open (3)/test/003.png
4,TEST_004,/content/drive/MyDrive/open (3)/test/004.png
...,...,...
787,TEST_787,/content/drive/MyDrive/open (3)/test/787.png
788,TEST_788,/content/drive/MyDrive/open (3)/test/788.png
789,TEST_789,/content/drive/MyDrive/open (3)/test/789.png
790,TEST_790,/content/drive/MyDrive/open (3)/test/790.png


In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
sample_submission = pd.read_csv(dataset_dir + 'sample_submission.csv')

sample_submission['label'] = preds

# sample_submission.loc[sample_submission['label'] == 0, 'label'] = '가구수정'
# sample_submission.loc[sample_submission['label'] == 1, 'label'] = '걸레받이수정'
# sample_submission.loc[sample_submission['label'] == 2, 'label'] = '곰팡이'
# sample_submission.loc[sample_submission['label'] == 3, 'label'] = '꼬임'
# sample_submission.loc[sample_submission['label'] == 4, 'label'] = '녹오염'
# sample_submission.loc[sample_submission['label'] == 5, 'label'] = '들뜸'
# sample_submission.loc[sample_submission['label'] == 6, 'label'] = '면불량'
# sample_submission.loc[sample_submission['label'] == 7, 'label'] = '몰딩수정'
# sample_submission.loc[sample_submission['label'] == 8, 'label'] = '반점'
# sample_submission.loc[sample_submission['label'] == 9, 'label'] = '석고수정'
# sample_submission.loc[sample_submission['label'] == 10, 'label'] = '오염'
# sample_submission.loc[sample_submission['label'] == 11, 'label'] = '오타공'
# sample_submission.loc[sample_submission['label'] == 12, 'label'] = '울음'
# sample_submission.loc[sample_submission['label'] == 13, 'label'] = '이음부불량'
# sample_submission.loc[sample_submission['label'] == 14, 'label'] = '창틀,문틀수정'
# sample_submission.loc[sample_submission['label'] == 15, 'label'] = '터짐'
# sample_submission.loc[sample_submission['label'] == 16, 'label'] = '틈새과다'
# sample_submission.loc[sample_submission['label'] == 17, 'label'] = '피스'
# sample_submission.loc[sample_submission['label'] == 18, 'label'] = '훼손'

sample_submission.to_csv('./baseline_solve.csv', index=False)
sample_submission